# APPENDIX - Rule-based prediction for Category

In [8]:
# Import relevant libraries
import pandas as pd
import numpy as np
import re

In [2]:
# read input file into dataframe

data=pd.read_csv('Full Data + Tagged Product Combined.csv')
data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id,product_id-2,product_color_id,attribute_name,attribute_value,file
0,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,LWAX0056,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,2019-10-06 15:31:31.730524+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/les-second-me...,NaN,{},NaN,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
1,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,LWAX0056,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,2019-10-06 15:31:31.730000+00:00,2020-04-06 23:19:53.216000+00:00,2020-04-06 23:19:53.216000+00:00,https://frame-store.com/products/les-second-me...,NaN,[],185.0,01DPGV4YRP3Z8J85DASGZ1Y99W,01DPGVGBK6YGNYGNF2S6FSH02T,style,Casual,initial_tags
2,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,491075,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,2019-11-11 22:22:21.664000+00:00,2020-03-25 23:24:44.823000+00:00,2020-03-23 21:06:15.953000+00:00,https://bananarepublic.gap.com/browse/product....,Everything you love about our original Madison...,[],431.0,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
3,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,491075,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,2019-11-11 22:22:21.664425+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Everything you love about our original Madison...,{},NaN,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,01DSE8ZG8Y3FR8KWE2TY1QDWBF,shoe_width,Medium,initial_tags
4,01E2C3YN4KQ36A0REWZJ89ZN73,FREDA SALVADOR,5229129,Ace Bootie,Edgy style and expert craftsmanship combine on...,Unknown,2020-03-01 22:37:32.169000+00:00,2020-04-15 21:46:03.512000+00:00,2020-03-18 23:00:31.558000+00:00,https://shop.nordstrom.com/s/freda-salvador-ac...,"True to size.\n2 1/4"" (57mm) heel (size 8.5)\n...",[],1051.0,01E2C3YN4KQ36A0REWZJ89ZN73,01E2C3YN56ZCJ8TN45V3EC8CPS,Primary Color,Blacks,initial_tags


In [3]:
# check null values
data.isnull().sum()

product_id                 0
brand                      0
mpn                        0
product_full_name          0
description            18496
brand_category          5870
created_at                 0
updated_at                 0
deleted_at             94172
brand_canonical_url       34
details                34370
labels                     0
bc_product_id          75961
product_id-2           44105
product_color_id       44105
attribute_name         44105
attribute_value        44105
file                   44105
dtype: int64

In [5]:
# drop unnecessary columns
data.drop(columns=['mpn', 'created_at', 'updated_at', 'deleted_at',
                'brand_canonical_url', 'bc_product_id','product_id-2','product_color_id','file'],inplace=True)

In [6]:
data.head()

,product_id,brand,product_full_name,description,brand_category,details,labels,attribute_name,attribute_value
0,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,NaN,{},style,Casual
1,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,NaN,[],style,Casual
2,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,Everything you love about our original Madison...,[],shoe_width,Medium
3,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,Everything you love about our original Madison...,{},shoe_width,Medium
4,01E2C3YN4KQ36A0REWZJ89ZN73,FREDA SALVADOR,Ace Bootie,Edgy style and expert craftsmanship combine on...,Unknown,"True to size.\n2 1/4"" (57mm) heel (size 8.5)\n...",[],Primary Color,Blacks
...,...,...,...,...,...,...,...,...,...
203113,01DPETKRJG1XH8XZESV7JSF4VP,J.Crew,Riley sandals in sunwashed pink patent leather,Our design team created this strappy sandal sp...,shoes,NaN,"{""Needs Attributes""}",NaN,NaN
203114,01DSE9X9C73BXXGXNAPMKGJTD1,Banana Republic,Glitz Short Necklace,"Dress it up or dress it down, our jewelry coll...",Unknown,"Dress it up or dress it down, our jewelry coll...","{""Needs Review""}",NaN,NaN
203115,01DSP4DTZF3P5EF1RR53AZ2TP2,Shinola,Vinton Stainless Steel Bracelet Watch,From the Vinton Collection. Featuring a matte ...,"JewelryAccessories/Watches/ForHim,TheMensStore...",Argonite 715 Swiss quartz movement\nPolished s...,"{""Needs Review""}",NaN,NaN
203116,01DPH1M5PXCK323CFYDMMJ4P9C,Sole Society,Nadina,Details \nSlide into style with this trend-...,Shoes,NaN,"{""Needs Review""}",NaN,NaN


### Regex to identify Category: Occasion

In [30]:
pattern=r'Casual|Weekend|Day\s?(?:to)\s?Night|Night\s?out|work(?:out)'


data['occasion']=np.where(data.brand.str.contains(pattern,flags=re.IGNORECASE),True,
                         np.where(data.product_full_name.str.contains(pattern,flags=re.IGNORECASE),True,
                         np.where(data.description.str.contains(pattern,flags=re.IGNORECASE),True,
                         np.where(data.brand_category.str.contains(pattern,flags=re.IGNORECASE),True,
                         np.where(data.details.str.contains(pattern,flags=re.IGNORECASE),True,False)))))


### Regex to identify Category: Style

In [17]:
pattern2=r'Androgynous|Athleisure|Boho|Business\s?(?:Casual)|Classic|Edgy|Glam|Modern|Retro|Romantic'


data['style']=np.where(data.brand.str.contains(pattern2,flags=re.IGNORECASE),True,
                         np.where(data.product_full_name.str.contains(pattern2,flags=re.IGNORECASE),True,
                         np.where(data.description.str.contains(pattern2,flags=re.IGNORECASE),True,
                         np.where(data.brand_category.str.contains(pattern2,flags=re.IGNORECASE),True,
                         np.where(data.details.str.contains(pattern2,flags=re.IGNORECASE),True,False)))))


### Regex to identify Category: Fit

In [19]:
pattern3=r'Fitted|Tailored|Semi\s?-?\s?Fitted|Straight\s?|Regular|Relaxed|Oversized'

data['fit']=np.where(data.brand.str.contains(pattern3,flags=re.IGNORECASE),True,
                         np.where(data.product_full_name.str.contains(pattern3,flags=re.IGNORECASE),True,
                         np.where(data.description.str.contains(pattern3,flags=re.IGNORECASE),True,
                         np.where(data.brand_category.str.contains(pattern3,flags=re.IGNORECASE),True,
                         np.where(data.details.str.contains(pattern3,flags=re.IGNORECASE),True,False)))))


### Regex to identify Category: Color

In [22]:
pattern4=r'Beiges|Blacks|Blues|Browns|Burgundies|Golds|Grays|Greens|Multi|Navy|Neutrals|Oranges|Pinks|Purples\
            |Reds|Silvers|Teal|Whites|Yellows'

data['color']=np.where(data.brand.str.contains(pattern4,flags=re.IGNORECASE),True,
                         np.where(data.product_full_name.str.contains(pattern4,flags=re.IGNORECASE),True,
                         np.where(data.description.str.contains(pattern4,flags=re.IGNORECASE),True,
                         np.where(data.brand_category.str.contains(pattern4,flags=re.IGNORECASE),True,
                         np.where(data.details.str.contains(pattern4,flags=re.IGNORECASE),True,False)))))


In [33]:
data['labels'] = data.apply(lambda x: list([x['occasion'],x['style'],x['fit'],x['color']]),axis=1)
data

,product_id,brand,product_full_name,description,brand_category,details,labels,attribute_name,attribute_value,occasion,style,fit,color
0,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,NaN,"[True, True, True, True]",style,Casual,True,True,True,True
1,01DPGV4YRP3Z8J85DASGZ1Y99W,Frame,Les Second - Medium--NOIR,"Minimal, Modern Styling Meets Refined Luxury I...",Accessories,NaN,"[True, True, True, True]",style,Casual,True,True,True,True
2,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,Everything you love about our original Madison...,"[False, True, False, False]",shoe_width,Medium,False,True,False,False
3,01DSE8Z2ZDAZKZ2SKCS1E3B3HK,Banana Republic,Madison 12-Hour Loafer Pump,Everything you love about our original Madison...,Unknown,Everything you love about our original Madison...,"[False, True, False, False]",shoe_width,Medium,False,True,False,False
4,01E2C3YN4KQ36A0REWZJ89ZN73,FREDA SALVADOR,Ace Bootie,Edgy style and expert craftsmanship combine on...,Unknown,"True to size.\n2 1/4"" (57mm) heel (size 8.5)\n...","[False, True, False, False]",Primary Color,Blacks,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203113,01DPETKRJG1XH8XZESV7JSF4VP,J.Crew,Riley sandals in sunwashed pink patent leather,Our design team created this strappy sandal sp...,shoes,NaN,"[True, True, True, True]",NaN,NaN,True,True,True,True
203114,01DSE9X9C73BXXGXNAPMKGJTD1,Banana Republic,Glitz Short Necklace,"Dress it up or dress it down, our jewelry coll...",Unknown,"Dress it up or dress it down, our jewelry coll...","[False, False, False, False]",NaN,NaN,False,False,False,False
203115,01DSP4DTZF3P5EF1RR53AZ2TP2,Shinola,Vinton Stainless Steel Bracelet Watch,From the Vinton Collection. Featuring a matte ...,"JewelryAccessories/Watches/ForHim,TheMensStore...",Argonite 715 Swiss quartz movement\nPolished s...,"[False, True, False, False]",NaN,NaN,False,True,False,False
203116,01DPH1M5PXCK323CFYDMMJ4P9C,Sole Society,Nadina,Details \nSlide into style with this trend-...,Shoes,NaN,"[True, True, True, True]",NaN,NaN,True,True,True,True
